### Complex Confounding on IMDb data

In [ ]:
import os
import ast
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import norm
from scipy.special import expit

# ATE estimation
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from doubleml import DoubleMLData, DoubleMLPLR
from causalml.inference.meta import BaseSRegressor

# Custom modules
from utils.project import set_root
from utils.io import save_results, load_results
from visualization.plotting import plot_ate_estimates
from helpers.ae import fit_autoencoder

In [ ]:
# Set working directory and directory for saving results
set_root()
results_dir = "results/comparison_learners/imdb/complex"

# Define the model name
model_name = "densenet121-res224-all"  # Pretrained model name

In [ ]:
# Load the CSV file
file_path = "data/imdb/imdb_with_hidden_states_sentiment.csv"
df_imdb_sent_prepro = pd.read_csv(file_path)

# Convert the 'hidden_state' column from string to actual lists
df_imdb_sent_prepro['hidden_state'] = df_imdb_sent_prepro['hidden_state'].apply(ast.literal_eval)

# Expand the 'hidden_state' column into separate columns
hidden_state_df = pd.DataFrame(df_imdb_sent_prepro['hidden_state'].tolist())

# Rename the hidden state columns
hidden_state_df.columns = [f"hidden_state_{i}" for i in range(hidden_state_df.shape[1])]

# Concatenate the expanded hidden states with the original DataFrame (excluding original 'hidden_state' column)
df_imdb_prepro = pd.concat([df_imdb_sent_prepro.drop(columns=['hidden_state']), hidden_state_df], axis=1)

# Drop the 'text' column containing the review text
df_imdb = df_imdb_prepro.drop(columns=['text'])

# Depict the first few rows of the DataFrame
df_imdb.head()

In [7]:
# Further preprcessing the dataset for the simulation

# Subsample the dataset to a manageable size for simulation
n = 4000  # Number of samples to draw
df_imdb_sampled = df_imdb.sample(n=n, replace=False, random_state=42) 

# Extract sentiment labels and latent representations
sent_label = df_imdb_sampled['label']
imdb_latent_rep = df_imdb_sampled.drop(columns=['label'])

### Autoencoder Training on Pre-trained Representations (Used for Complex Confounding Simuation)

In [ ]:
# Train an autoencoder on the pre-trained representations (latent_features of the AE will be used to simulate confounding)
latent_dim = 5 # Dimensionality of the latent space
encoder, latent_features, scaler = fit_autoencoder(imdb_latent_rep, latent_dim=latent_dim, epochs=50, batch_size=32, random_seed=42)

# Obtain scaled imdb pre-trained representations 
imdb_latent_rep_scaled = scaler.transform(imdb_latent_rep)

### Complex Confounding Simulation and ATE Estimation

In [ ]:
# 1. Define Simulation Parameters
np.random.seed(42)  # For reproducibility
beta_true = 2.0     # True effect of A on Y
gammas_Y = np.random.uniform(-1, 0, latent_dim)   # Effect latent feature confounder on Y
gamma_A = np.random.normal(0, 1, latent_dim)  # Effect latent feature confounder on A

# 2. Specify general parameters for simulation
n_samples = latent_features.shape[0]
d_features = imdb_latent_rep_scaled.shape[1]
n_runs = 5  # Number of simulation runs
ci_alpha_level = 0.05  # Alpha level for 1-alpha confidence intervals
z_score = norm.ppf(1 - ci_alpha_level / 2) # Z-score for 1-alpha confidence intervals

# 3. Initialize Storage for Estimates and Confidence Intervals
methods = ['Naive', 'S-Learner (NN)', 'S-Learner (RF)', 'DML (NN)', 'DML (RF)'] 

estimates_dict_complex = {method: [] for method in methods}
cis_dict_complex = {method: {'lower': [], 'upper': []} for method in methods}

# 4. Simulation Loop
for run in range(n_runs):
    print(f"\n--- Simulation Run {run + 1} ---")
    # Set a unique seed for each run for variability
    seed = 42 + run
    np.random.seed(seed)
    
    # a. Compute Treatment Assignment Probability
    pA = expit(latent_features @ gamma_A)

    # b. Sample Treatment Assignment
    A = np.random.binomial(1, pA)

    # c. Generate Outcome Y
    noise = np.random.normal(loc=0, scale=1, size=n_samples)
    Y = beta_true * A + latent_features @ gammas_Y + noise
    
    # d. Package into DataFrame
    df = pd.DataFrame({
        'Y': Y,
        'A': A
    })
    
    # 4.1. Naive OLS (Unadjusted) using statsmodels
    X_naive = sm.add_constant(df['A']) 
    model_naive = sm.OLS(df['Y'], X_naive).fit()
    beta_naive = model_naive.params['A']
    se_naive = model_naive.bse['A']
    ci_lower_naive = beta_naive - z_score * se_naive
    ci_upper_naive = beta_naive + z_score * se_naive
    estimates_dict_complex['Naive'].append(beta_naive)
    cis_dict_complex['Naive']['lower'].append(ci_lower_naive)
    cis_dict_complex['Naive']['upper'].append(ci_upper_naive)
    print(f"Naive OLS: β = {beta_naive:.3f}, SE = {se_naive:.3f}")
    
    ## Different Other ATE Estimators

    # 4.2.1. S-Learner (NN)
    outcome_model_nn = Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPRegressor(
            hidden_layer_sizes=(100, 50),
            activation='relu',
            solver='adam',
            max_iter=500,
            random_state=seed
        ))
    ])

    try:
        s_learner_nn = BaseSRegressor(outcome_model_nn) 
        s_ate_nn, s_ci_lower_nn, s_ci_upper_nn = s_learner_nn.estimate_ate(imdb_latent_rep_scaled, A, Y, return_ci=True)
        estimates_dict_complex['S-Learner (NN)'].append(s_ate_nn[0])
        cis_dict_complex['S-Learner (NN)']['lower'].append(s_ci_lower_nn[0])
        cis_dict_complex['S-Learner (NN)']['upper'].append(s_ci_upper_nn[0])
        print(f"S-Learner (NN): β = {s_ate_nn[0]:.3f}")
    except Exception as e:
        print(f"Run {run+1}: S-Learner (NN) failed with error: {e}")
        estimates_dict_complex['S-Learner (NN)'].append(np.nan)
        cis_dict_complex['S-Learner (NN)']['lower'].append(np.nan)
        cis_dict_complex['S-Learner (NN)']['upper'].append(np.nan)
    
    # 4.2.2. S-Learner (RF) 
    outcome_model_rf = RandomForestRegressor(n_estimators=50, random_state=42)
    try:
        s_learner_rf = BaseSRegressor(outcome_model_rf) 
        s_ate_rf, s_ci_lower_rf, s_ci_upper_rf = s_learner_rf.estimate_ate(imdb_latent_rep_scaled, A, Y, return_ci=True)
        estimates_dict_complex['S-Learner (RF)'].append(s_ate_rf[0])
        cis_dict_complex['S-Learner (RF)']['lower'].append(s_ci_lower_rf[0])
        cis_dict_complex['S-Learner (RF)']['upper'].append(s_ci_upper_rf[0])
        print(f"S-Learner (RF): β = {s_ate_rf[0]:.3f}")
    except Exception as e:
        print(f"Run {run+1}: S-Learner (RF) failed with error: {e}")
        estimates_dict_complex['S-Learner (RF)'].append(np.nan)
        cis_dict_complex['S-Learner (RF)']['lower'].append(np.nan)
        cis_dict_complex['S-Learner (RF)']['upper'].append(np.nan)

    # 4.3. DoubleML with Neural Network Nuisance and Random Forest Estimators
    # Convert imdb_latent_rep to DataFrame
    X_dml_df = pd.DataFrame(
        imdb_latent_rep_scaled,  # Use scaled features
        columns=[f"feat_{i}" for i in range(d_features)]
    )
    
    # Add outcome and treatment to DoubleMLData via column names
    X_dml_df['Y'] = df['Y']
    X_dml_df['A'] = df['A']

    # Create DoubleMLData
    data_dml = DoubleMLData(X_dml_df, "Y", "A")

    # 4.3.1. DoubleML with Neural Network Nuisance Estimators
    try:
        # Define nuisance models with a neural network and logistic regression
        ml_g_nn = Pipeline([
            ('scaler', StandardScaler()),
            ('mlp', MLPRegressor(
                hidden_layer_sizes=(100, 50),
                activation='relu',
                solver='adam',
                max_iter=500,
                random_state=seed
            ))
        ])
        
        ml_m_log = LogisticRegression()
    
        # Instantiate and fit DoubleMLPLR
        dml_plr_nn = DoubleMLPLR(data_dml, ml_g_nn, ml_m_log, n_folds=2)
        dml_plr_nn.fit()
        beta_dml_nn = dml_plr_nn.coef[0]
        se_dml_nn = dml_plr_nn.se[0]
        estimates_dict_complex['DML (NN)'].append(beta_dml_nn)
        # 95% Confidence Interval
        ci_lower_dml_nn = beta_dml_nn - z_score * se_dml_nn
        ci_upper_dml_nn = beta_dml_nn + z_score * se_dml_nn
        cis_dict_complex['DML (NN)']['lower'].append(ci_lower_dml_nn)
        cis_dict_complex['DML (NN)']['upper'].append(ci_upper_dml_nn)
        print(f"DML (NN): β = {beta_dml_nn:.3f}, SE = {se_dml_nn:.3f}")
    except Exception as e:
        print(f"Run {run+1}: DML (NN) failed with error: {e}")
        estimates_dict_complex['DML'].append(np.nan)
        cis_dict_complex['DML (NN)']['lower'].append(np.nan)
        cis_dict_complex['DML (NN)']['upper'].append(np.nan)

    # 4.3.2. DoubleML with Random Forest Nuisance Estimators

    try:
        # Define nuisance models with neural networks
        ml_g_rf = RandomForestRegressor(n_estimators=50, random_state=42)
        ml_m_rf = RandomForestClassifier(n_estimators=50, random_state=42)
    
        # Instantiate and fit DoubleMLPLR
        dml_plr_rf = DoubleMLPLR(data_dml, ml_g_rf, ml_m_rf, n_folds=2) 
        dml_plr_rf.fit()
        beta_dml_rf = dml_plr_rf.coef[0]
        se_dml_rf = dml_plr_rf.se[0]
        estimates_dict_complex['DML (RF)'].append(beta_dml_rf)
        # 95% Confidence Interval
        ci_lower_dml_rf = beta_dml_rf - z_score * se_dml_rf
        ci_upper_dml_rf = beta_dml_rf + z_score * se_dml_rf
        cis_dict_complex['DML (RF)']['lower'].append(ci_lower_dml_rf)
        cis_dict_complex['DML (RF)']['upper'].append(ci_upper_dml_rf)
        print(f"DML (RF): β = {beta_dml_rf:.3f}, SE = {se_dml_rf:.3f}")
    except Exception as e:
        print(f"Run {run+1}: DML failed with error: {e}")
        estimates_dict_complex['DML (RF)'].append(np.nan)
        cis_dict_complex['DML (RF)']['lower'].append(np.nan)
        cis_dict_complex['DML (RF)']['upper'].append(np.nan)

In [6]:
# 5. Create a directory for the experiment and save the results
experiment_name = "exp_results"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_dir = os.path.join(results_dir, model_name, experiment_name, timestamp)
save_results(experiment_dir, estimates_dict_complex, cis_dict_complex)

### Plotting Results

In [4]:
# Load the results from the previous experiment
estimates_dict_complex, cis_dict_complex = load_results(experiment_dir)

In [ ]:
# Plot ATE estimates with confidence intervals
plot_ate_estimates(
    estimates_dict=estimates_dict_complex,
    cis_dict=cis_dict_complex,
    plot_name="ate_estimates_complex_conf_imdb",
    save_dir=experiment_dir,
    ate_true=2.0,
    n_runs=5,
    vert_diff=0.1,
    verbose=True
)